In [7]:
using Random

In [19]:
function get_instancia(n)
    return rand(1:100, n)
end

function get_permutacoes(instancia)
    return [shuffle(instancia) for _=1:20]
end

get_permutacoes (generic function with 1 method)

In [211]:
function get_custo(instancia, tamanho, custo_total)
    if tamanho == 1
        return instancia[tamanho], 0
    else
        espera, custo_total = get_custo(instancia[1:tamanho-1], tamanho-1, custo_total)
        custo = instancia[tamanho] + espera
        return custo, custo_total + custo
    end
end

get_custo (generic function with 2 methods)

In [212]:
instancia_inicial = get_instancia(100);
permutacoes_iniciais = get_permutacoes(instancia_inicial);
for permutacao in permutacoes_iniciais
    custos = zeros(0)
    _, custo = get_custo(permutacao, 100, 0)
    println(custo)
end

270955
266193
259439
269147
261575
256486
282666
259163
262284
262428
272882
267868
267884
268151
264529
270346
261689
264901
267847
254196


In [60]:
permutacoes

10×20 Matrix{Int64}:
 32  65   1   1  75  75   1  31  30  …  30  31  32  31  75  58   1  31  57
 32  31  30  58  30  65  32  40  30     58  31  40  31  31  58  32  30  32
 57  31  31  17  65  58  57  75  32     57  58  75  65  57  65  17  58   1
 32  30  31  40  30  30  65  58  58      1  65  30  40  65   1  65  17  30
 32   1  58  40  32  57  31  65  30     32  58   1  17  32  30  32  65  31
 17  57   1  17  65  17  17  32  17  …  30  65  40  32  57  31  31  32  32
 75  30   1  65  31  30  32  30  31     40  65  30  32  17  65  32  30  57
 65  57  57  32  17  75   1  40  58     57   1  57  32  40  65  31  65  31
 17  31  75  30  57  17   1  31  58     30   1  65  75  57  58  30  58  58
 30  75  57  65  17  75  58  17   1     58  30  65   1  30  17  40  40  31